In [1]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def save_checkpoint(epoch, model, optimizer):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_neuralode_cifar100.pth.tar'
    torch.save(state, filename)

class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [3]:
# 数据集读取

epochs = 200
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01
mean = [0.5070751592371323, 0.48654887331495095, 0.4409178433670343]
std = [0.2673342858792401, 0.2564384629170883, 0.27615047132568404]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = torchvision.datasets.CIFAR100(root='data', train=True, download=False, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR100(root='data', train=False, download=False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)


In [4]:
def ode_solve(z0, t0, t1, f):
    """
    Simplest Euler ODE initial value solver
    """
    h_max = 0.05
    n_steps = math.ceil((abs(t1 - t0)/h_max).max().item())  #向上取整

    h = (t1 - t0)/n_steps
    t = t0
    z = z0

    for i_step in range(n_steps):
        z = z + h * f(z, t)
        t = t + h
    return z

In [5]:
class ODEF(nn.Module):
    def forward_with_grad(self, z, t, grad_outputs):
        """Compute f and a df/dz, a df/dp, a df/dt"""
        batch_size = z.shape[0]  #矩阵的列数

        out = self.forward(z, t)

        a = grad_outputs
        adfdz, adfdt, *adfdp = torch.autograd.grad(
            (out,), (z, t) + tuple(self.parameters()), grad_outputs=(a),
            allow_unused=True, retain_graph=True
        )    #自动求导
        # grad method automatically sums gradients for batch items, we have to expand them back 
        if adfdp is not None:
            adfdp = torch.cat([p_grad.flatten() for p_grad in adfdp]).unsqueeze(0)  #关于p的参数转化为一维数组并加一个参数
            adfdp = adfdp.expand(batch_size, -1) / batch_size   #加batch_size个行 再除以batch_size
        if adfdt is not None:
            adfdt = adfdt.expand(batch_size, 1) / batch_size
        return out, adfdz, adfdt, adfdp

    def flatten_parameters(self):
        p_shapes = []
        flat_parameters = []
        for p in self.parameters():
            p_shapes.append(p.size())
            flat_parameters.append(p.flatten())
        return torch.cat(flat_parameters)   #所有向量放到一个维度

In [6]:
class ODEAdjoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, z0, t, flat_parameters, func):
        assert isinstance(func, ODEF)      #判断func是否在ODEF类
        bs, *z_shape = z0.size()   #bs是有几行(数据维度),z_shape有几列(数据数量)
        time_len = t.size(0)  #有几行

        with torch.no_grad():  #即使输入求导，输出不求导
            z = torch.zeros(time_len, bs, *z_shape).to(z0)   #三个维度, 时间*数据
            z[0] = z0
            for i_t in range(time_len - 1):
                z0 = ode_solve(z0, t[i_t], t[i_t+1], func)
                z[i_t+1] = z0
        ctx.func = func
        ctx.save_for_backward(t, z.clone(), flat_parameters)  #为了反向传播保留input全部信息
        return z

    @staticmethod
    def backward(ctx, dLdz):
        """
        dLdz shape: time_len, batch_size, *z_shape
        """
        func = ctx.func
        t, z, flat_parameters = ctx.saved_tensors   #被储存的参数
        time_len, bs, *z_shape = z.size()    #z的时间，维度，数据量
        n_dim = np.prod(z_shape)      #np内元素的乘积 
        n_params = flat_parameters.size(0)  #参数数量，size[0]是有几行
        # Dynamics of augmented system to be calculated backwards in time
        def augmented_dynamics(aug_z_i, t_i):
            """
            tensors here are temporal slices
            t_i - is tensor with size: bs, 1
            aug_z_i - is tensor with size: bs, 2*n_dim + n_params + 1
            """
            #aug_z_i为原方程中的增广状态[z(t1),dL/dz(t1),0(\theta),-dL/dt]
            #t_i的维度？ time_len?
            z_i, a = aug_z_i[:, :n_dim], aug_z_i[:, n_dim:2*n_dim]  # ignore parameters and time  
            #z_i是z(t1)，a是dL/dz
            
            # Unflatten z and a
            z_i = z_i.view(bs, *z_shape)   #括号里面是想要的维度,torch.view()的作用
            a = a.view(bs, *z_shape)
            with torch.set_grad_enabled(True):
                #detach_()将计算图中节点转为叶子节点，也就是将节点.grad_fn设置为none，这样detach_()的前一个节点就不会再与当前变量连接
                t_i = t_i.detach().requires_grad_(True) 
                z_i = z_i.detach().requires_grad_(True)
                func_eval, adfdz, adfdt, adfdp = func.forward_with_grad(z_i, t_i, grad_outputs=a)  # bs, *z_shape
                #func_eval是输出, 其余为a乘以导数
                adfdz = adfdz.to(z_i) if adfdz is not None else torch.zeros(bs, *z_shape).to(z_i)  #torch.to()
                adfdp = adfdp.to(z_i) if adfdp is not None else torch.zeros(bs, n_params).to(z_i)
                adfdt = adfdt.to(z_i) if adfdt is not None else torch.zeros(bs, 1).to(z_i)

            # Flatten f and adfdz
            func_eval = func_eval.view(bs, n_dim)
            adfdz = adfdz.view(bs, n_dim) 
            return torch.cat((func_eval, -adfdz, -adfdp, -adfdt), dim=1)  #输出新的增广状态

        dLdz = dLdz.view(time_len, bs, n_dim)  # flatten dLdz for convenience

        with torch.no_grad():
            ## Create placeholders for output gradients
            # Prev computed backwards adjoints to be adjusted by direct gradients
            adj_z = torch.zeros(bs, n_dim).to(dLdz)
            adj_p = torch.zeros(bs, n_params).to(dLdz)
            # In contrast to z and p we need to return gradients for all times
            adj_t = torch.zeros(time_len, bs, 1).to(dLdz)

            for i_t in range(time_len-1, 0, -1):     #反向传播
                z_i = z[i_t]
                t_i = t[i_t]
                f_i = func(z_i, t_i).view(bs, n_dim)

                # Compute direct gradients
                dLdz_i = dLdz[i_t]
                dLdt_i = torch.bmm(torch.transpose(dLdz_i.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0] 
                #bmm三维tensor乘积 第一个维度一样 后两个维度矩阵乘积
                #torch.transpose(a,1,2)是交换a第2和第3维度
                #torch.unsqueeze(-1)是在最后一个维度上加一个
                #dL/dt = dL/dz*dz/dt 在对应的时间节点求

                # Adjusting adjoints with direct gradients
                adj_z += dLdz_i    #z的伴随就是dL/dz  根据节点调整 图上的
                adj_t[i_t] = adj_t[i_t] - dLdt_i   

                # Pack augmented variable
                aug_z = torch.cat((z_i.view(bs, n_dim), adj_z, torch.zeros(bs, n_params).to(z), adj_t[i_t]), dim=-1) #按照bs拼接

                # Solve augmented system backwards
                aug_ans = ode_solve(aug_z, t_i, t[i_t-1], augmented_dynamics)
                
                # Unpack solved backwards augmented system
                adj_z[:] = aug_ans[:, n_dim:2*n_dim]
                adj_p[:] += aug_ans[:, 2*n_dim:2*n_dim + n_params]   #不需要调整，一直加
                adj_t[i_t-1] = aug_ans[:, 2*n_dim + n_params:]
                #得出来的结果分配到新的增广状态

                del aug_z, aug_ans

            ## Adjust 0 time adjoint with direct gradients
            # Compute direct gradients 

            dLdz_0 = dLdz[0]
            dLdt_0 = torch.bmm(torch.transpose(dLdz_0.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0]

            # Adjust adjoints
            adj_z += dLdz_0
            adj_t[0] = adj_t[0] - dLdt_0
        return adj_z.view(bs, *z_shape), adj_t, adj_p, None    #得到了到t0的增广状态

In [7]:
class NeuralODE(nn.Module):
    def __init__(self, func):
        super(NeuralODE, self).__init__()  #调用父类
        assert isinstance(func, ODEF)
        self.func = func

    def forward(self, z0, t=Tensor([0., 1.]), return_whole_sequence=False):
        t = t.to(z0)
        z = ODEAdjoint.apply(z0, t, self.func.flatten_parameters(), self.func)   #只有前向传播
        if return_whole_sequence:
            return z
        else:
            return z[-1]

In [8]:
def norm(dim):
    return nn.BatchNorm2d(dim)  #norm为batchnormalization函数

def conv3x3(in_feats, out_feats, stride=1):
    return nn.Conv2d(in_feats, out_feats, kernel_size=3, stride=stride, padding=1, bias=False) #二维数据的卷积操作

def add_time(in_tensor, t):
    bs, c, w, h = in_tensor.shape
    return torch.cat((in_tensor, t.expand(bs, 1, w, h)), dim=1)  #增加时间变量

In [9]:
class ConvODEF(ODEF):
    def __init__(self, dim):
        super(ConvODEF, self).__init__()
        self.conv1 = conv3x3(dim + 1, dim)
        self.norm1 = norm(dim)
        self.conv2 = conv3x3(dim + 1, dim)
        self.norm2 = norm(dim)

    def forward(self, x, t):
        xt = add_time(x, t)
        h = self.norm1(torch.relu(self.conv1(xt)))
        ht = add_time(h, t)
        dxdt = self.norm2(torch.relu(self.conv2(ht)))
        return dxdt

In [10]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [11]:
class odenet(nn.Module):
    def __init__(self, ResidualBlock, ConvODEF, num_classes=100):
        super(odenet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        func = ConvODEF(64)
        self.layer1 = NeuralODE(func)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)    #核为4，步长为4
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def odeNet18():

    return odenet(ResidualBlock, ConvODEF)

In [ ]:
# checkpoint = None
checkpoint = 'checkpoint_neuralode_cifar100.pth.tar'
print_freq = 20

def main():
    """
    Training.
    """
    
    global start_epoch, classes, epoch, checkpoint
    
    # 初始化模型
    
    if checkpoint is None:
        start_epoch = 0
        model = odeNet18()
        optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    else:
        checkpoint = torch.load(checkpoint, map_location = 'cuda')
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(start_epoch, epochs):
        
        if epoch == 100:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.1, momentum=0.9, weight_decay=5e-4)
        if epoch == 150:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.01, momentum=0.9, weight_decay=5e-4)
        
        train(train_loader = train_loader,
             model = model,
             criterion=criterion,
             optimizer=optimizer,
             epoch=epoch)
        save_checkpoint(epoch, model, optimizer)
        evaluate(test_loader, model)
        

def train(train_loader, model, criterion, optimizer, epoch):
    
    model = model.train()
    
    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()
    total = 0
    correct = 0
    for i, data in enumerate(train_loader, 0):
        length = len(train_loader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward + backward
        outputs = model(inputs)
        loss = criterion(outputs, labels).to(device)
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), inputs.size(0))
        batch_time.update(time.time() - start)
        start = time.time()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1 = open("train_odenet18_cifar100.txt", "a")
            f1.write('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t\n'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1.close()
    print(correct/total)
    f2 = open("train_acc_odenet18_cifar100.txt", "a")
    f2.write('odenet_cifar100测试分类准确率为：%.3f%%\n' % (100 * correct / total))
    f2.close()
def evaluate(test_loader, model):
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # 取得分最高的那个类 (outputs.data的索引号)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('测试分类准确率为：%.3f%%' % (100 * correct / total))
        f3 = open("test_acc_odenet18_cifar100.txt", "a")
        f3.write('odenet_cifar100测试分类准确率为：%.3f%%\n' % (100 * correct / total))
        f3.close()
        acc = 100. * correct / total
        best_acc = 85
        if acc > best_acc:
            f3 = open("best_acc_odenet18_cifar100.txt", "w")
            f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
            f3.close()
            best_acc = acc
if __name__ == '__main__':
    main()


Loaded checkpoint from epoch 191.

Epoch: [191][0/391]	Batch Time 5.057 (5.057)	Data Time 0.000 (0.000)	Loss 0.0060 (0.0060)	
Epoch: [191][20/391]	Batch Time 1.265 (1.435)	Data Time 0.000 (0.000)	Loss 0.0069 (0.0067)	
Epoch: [191][40/391]	Batch Time 1.250 (1.347)	Data Time 0.000 (0.000)	Loss 0.0085 (0.0070)	
Epoch: [191][60/391]	Batch Time 1.265 (1.318)	Data Time 0.000 (0.000)	Loss 0.0043 (0.0070)	
Epoch: [191][80/391]	Batch Time 1.258 (1.303)	Data Time 0.000 (0.000)	Loss 0.0162 (0.0071)	
Epoch: [191][100/391]	Batch Time 1.265 (1.294)	Data Time 0.000 (0.000)	Loss 0.0071 (0.0072)	
Epoch: [191][120/391]	Batch Time 1.265 (1.289)	Data Time 0.000 (0.000)	Loss 0.0051 (0.0070)	
Epoch: [191][140/391]	Batch Time 1.268 (1.285)	Data Time 0.000 (0.000)	Loss 0.0060 (0.0069)	
Epoch: [191][160/391]	Batch Time 1.279 (1.283)	Data Time 0.000 (0.000)	Loss 0.0085 (0.0071)	
Epoch: [191][180/391]	Batch Time 1.263 (1.281)	Data Time 0.000 (0.000)	Loss 0.0067 (0.0070)	
Epoch: [191][200/391]	Batch Time 1.274 (

Epoch: [195][140/391]	Batch Time 1.400 (1.422)	Data Time 0.000 (0.000)	Loss 0.0074 (0.0067)	
Epoch: [195][160/391]	Batch Time 1.401 (1.420)	Data Time 0.000 (0.000)	Loss 0.0043 (0.0069)	
Epoch: [195][180/391]	Batch Time 1.399 (1.418)	Data Time 0.000 (0.000)	Loss 0.0068 (0.0069)	
Epoch: [195][200/391]	Batch Time 1.401 (1.416)	Data Time 0.000 (0.000)	Loss 0.0080 (0.0069)	
Epoch: [195][220/391]	Batch Time 1.399 (1.415)	Data Time 0.000 (0.000)	Loss 0.0056 (0.0069)	
Epoch: [195][240/391]	Batch Time 1.399 (1.414)	Data Time 0.000 (0.000)	Loss 0.0117 (0.0069)	
Epoch: [195][260/391]	Batch Time 1.400 (1.413)	Data Time 0.000 (0.000)	Loss 0.0045 (0.0069)	
Epoch: [195][280/391]	Batch Time 1.269 (1.409)	Data Time 0.000 (0.000)	Loss 0.0062 (0.0068)	
Epoch: [195][300/391]	Batch Time 1.261 (1.399)	Data Time 0.000 (0.000)	Loss 0.0055 (0.0068)	
Epoch: [195][320/391]	Batch Time 1.262 (1.391)	Data Time 0.000 (0.000)	Loss 0.0055 (0.0069)	
Epoch: [195][340/391]	Batch Time 1.262 (1.383)	Data Time 0.000 (0.000)

Epoch: [199][280/391]	Batch Time 1.254 (1.272)	Data Time 0.000 (0.000)	Loss 0.0052 (0.0066)	
Epoch: [199][300/391]	Batch Time 1.256 (1.271)	Data Time 0.000 (0.000)	Loss 0.0046 (0.0066)	
Epoch: [199][320/391]	Batch Time 1.253 (1.270)	Data Time 0.000 (0.000)	Loss 0.0062 (0.0066)	
Epoch: [199][340/391]	Batch Time 1.270 (1.270)	Data Time 0.000 (0.000)	Loss 0.0060 (0.0066)	
Epoch: [199][360/391]	Batch Time 1.270 (1.269)	Data Time 0.000 (0.000)	Loss 0.0039 (0.0066)	
